In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [2]:
cookies = {'session':'53616c7465645f5ff4b35a6eb924fde728d21409670086f7994aae3d5f27a69035f4466a65096035e2fe25fc34f889af'}
def get_data(qno):
    response = requests.get(f'https://adventofcode.com/2020/day/{str(qno)}/input', cookies = cookies)
    data = response.text
    return data

# Q1. 
Specifically, they need you to find the two entries that sum to 2020 and then multiply those two numbers together.

In [ ]:
# response

# response.headers

In [8]:
data = [int(x) for x in get_data(1).split()]

from itertools import combinations

a, b = [pair for pair in combinations(data, 2) if sum(pair) == 2020][0]
print(a * b)

326211


## In your expense report, what is the product of the three entries that sum to 2020?

In [6]:
a, b, c = [triple for triple in combinations(data, 3) if sum(triple) == 2020][0]
print(a * b * c)

131347190


# Q2. 


In [10]:
data = get_data(2)
data = data.split('\n')

# data

valid = 0
for rule in data[:-1]:
    bounds, char, _, pwd = [x.strip() for x in re.split(':| ', rule)]
    lb, ub = [int(x) for x in bounds.split('-')] 
    count = list(pwd).count(char)
    if  count <= ub and count >= lb:
        valid += 1

print(valid)

600


## New Policy:
Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

In [6]:
valid = 0
for i, rule in enumerate(data[:-1]):
    pos, char, _, pwd = [x.strip() for x in re.split(':| ', rule)]
    p1, p2 = [int(x)-1 for x in pos.split('-')] 
    if int(pwd[p1] == char) + int(pwd[p2] == char) == 1:
        valid += 1
print(valid)

245


# Q3. 
Starting at the top-left corner of your map and following a slope of right 3 and down 1, how many trees would you encounter?

In [7]:
data = get_data(3)
data = data.strip()

import numpy as np

n = len(data.split('\n'))
data = np.array(list(data.replace('\n','')))
data = data.reshape(n,-1)

count, i, j = 0, 0, 0 
d2 = data[:]

while i < data.shape[0]:
    if d2[i, j] == '#':
        count += 1
    i += 1
    j += 3
    if j >= d2.shape[1]:
        d2 = np.concatenate((d2, data), axis = 1)
        
print(count)

262


## Determine the number of trees you would encounter if, for each of the following slopes, you start at the top-left corner and traverse the map all the way to the bottom:

1. Right 1, down 1.
2. Right 3, down 1. (This is the slope you already checked.)
3. Right 5, down 1.
4. Right 7, down 1.
5. Right 1, down 2.

What do you get if you multiply together the number of trees encountered on each of the listed slopes?

In [8]:
ans = 1
slopes = [(1, 1),(3, 1), (5, 1), (7, 1), (1, 2)] # right and down
d2 = ''

for t in slopes:
    count, i, j = 0, 0, 0
    d2 = data[:]
    while i < data.shape[0]:
        if d2[i, j] == '#':
            count += 1
        j += t[0]
        i += t[1]

        if j >= data.shape[1]:
            d2 = np.concatenate((d2, data), axis = 1)
    
    ans *= count
        
print(ans)

2698900776


# Q4. 
The expected fields are as follows:

1. byr (Birth Year)
2. iyr (Issue Year)
3. eyr (Expiration Year)
4. hgt (Height)
5. hcl (Hair Color)
6. ecl (Eye Color)
7. pid (Passport ID)
7. cid (Country ID)
  
Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.  
Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

In [3]:
data = [x.split() for x in get_data(4).split('\n\n') if x!= '']

In [137]:
valid, valids = 0, [set(['eyr', 'hgt', 'hcl', 'pid', 'byr', 'ecl', 'iyr', 'cid'])]
valids += [set([i for i in valids[0] if i!= 'cid'])]

for passport in data:
    if set([item.split(':')[0] for item in passport]) in valids:
        valid += 1

print(valid)

260


## You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

- byr (Birth Year) - four digits; at least 1920 and at most 2002.
- iyr (Issue Year) - four digits; at least 2010 and at most 2020.
- eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
- hgt (Height) - a number followed by either cm or in:  
    If cm, the number must be at least 150 and at most 193.  
    If in, the number must be at least 59 and at most 76.  
- hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
- ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
- pid (Passport ID) - a nine-digit number, including leading zeroes.
- cid (Country ID) - ignored, missing or not.

Your job is to count the passports where all required fields are both present and valid according to the above rules.

In [5]:
fields = {
        'byr': lambda yr: 1920 <= int(yr) <= 2002,
        'iyr': lambda yr: 2010 <= int(yr) <= 2020,
        'eyr': lambda yr: 2020 <= int(yr) <= 2030,
        'hgt': lambda s: 150 <= int(s[:-2]) <= 193 if s[-2:] == 'cm' else 59 <= int(s[:-2]) <= 76 if s[-2:] == 'in' else False,
        'hcl': lambda hcl: re.search(r'^#(\d|[a-f]){6}$', hcl)!= None,
        'ecl': lambda ecl: ecl in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'],
        'pid': lambda n: n.isnumeric() and len(n) == 9
        }

ans = 0
vp2 = []
for passport in data:
    pp = {item.split(':')[0]: item.split(':')[1] for item in passport if item[:3]!='cid'}
    if (set(pp) == set(fields)) and sum([fields[f](pp[f]) for f in pp])==7:
        ans += 1 
        vp2.append(pp)
print(ans)

153


In [192]:
# requirements = {
#     'byr': r'(19[2-9]\d)|(200[0-2])',
#     'iyr': r'(201\d)|(2020)',
#     'eyr': r'(202\d)|(2030)',
#     'hgt': r'((1(([5-8]\d)|(9[0-3])))cm)|((59)|(6\d)|(7[0-6])in)', # problemmatic
#     'hcl': r'#(\d|[a-f]){6}',
#     'ecl': r'(amb|blu|brn|gry|grn|hzl|oth)',
#     'pid': r'^\d{9}$'
# }

# ans = 0
# vp = []
# for passport in data:
#     pp = {item.split(':')[0]: item.split(':')[1] for item in passport if item[:3]!= 'cid'}
#     if (set(pp) == set(requirements)) and sum([re.search(requirements[field], pp[field]) != None for field in pp])==7:
#         ans += 1  
#         vp.append(pp)

# print(ans)

# len(vp), len(vp2)
# for i in vp:
#     if i not in vp2:
#         print(dict(sorted(i.items()))) # invalid passports that regex didnt exclude

155
